# データサイエンス100本ノック 91~100

In [2]:
import polars as pl
from helper.polars import load_100knocks_data
pl.Config.set_fmt_str_lengths(100)
df_customer, df_category, df_product, df_receipt, df_store, df_geocode = load_100knocks_data()

## P-091

顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [3]:
df_tmp = (
df_customer
.join(
    df_receipt
    .group_by(pl.col.customer_id)
    .agg(sum_amount=pl.col.amount.sum()),
    on='customer_id',
    how='left'
)
)

flag = pl.col.sum_amount.is_null().cast(pl.UInt8)

df_result = df_tmp.filter(
    pl.int_range(pl.len()).shuffle().over(flag) < pl.len().over(flag).min()
)

(
df_result
.group_by(flag)
.agg(count=pl.col.customer_id.count())
)

sum_amount,count
u8,u32
0,8306
1,8306


## P-092

顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [4]:
df_gender_std = df_customer.select('gender_cd', 'gender').unique()
df_gender_std

gender_cd,gender
str,str
"""9""","""不明"""
"""1""","""女性"""
"""0""","""男性"""


In [5]:
df_customer_std = df_customer.select(
    'customer_id',
    'customer_name',
    'gender_cd',
    'birth_day',
    'age',
    'postal_cd',
    'application_store_cd',
    'application_date',
    'status_cd'
)
df_customer_std.head()

customer_id,customer_name,gender_cd,birth_day,age,postal_cd,application_store_cd,application_date,status_cd
str,str,str,str,i64,str,str,i64,str
"""CS021313000114""","""大野 あや子""","""1""","""1981-04-29""",37,"""259-1113""","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071""","""六角 雅彦""","""9""","""1952-04-01""",66,"""136-0076""","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172""","""宇多田 貴美子""","""1""","""1976-10-04""",42,"""151-0053""","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001""","""堀井 かおり""","""1""","""1933-03-27""",86,"""245-0016""","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145""","""田崎 美紀""","""1""","""1995-03-29""",24,"""144-0055""","""S13001""",20170605,"""6-20090929-2"""


## P-093

商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [6]:
df_product_full = (
df_product.join(
    df_category,
    on='category_small_cd',
    how='inner'
)
.select(
    'product_cd',
    'category_major_cd',
    'category_major_name',
    'category_medium_cd',
    'category_medium_name',
    'category_small_cd',
    'category_small_name',
    'unit_price',
    'unit_cost'
)
)
df_product_full.head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,str,str,str,str,str,str,i64,i64
"""P040101001""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",198,149
"""P040101002""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",218,164
"""P040101003""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",230,173
"""P040101004""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",248,186
"""P040101005""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",268,201


## P-094

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [7]:
df_product_full.write_csv('./data/product_full.csv')

## P-095

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [8]:
import io
buf = io.BytesIO()
df_product_full.write_csv(buf)
with open('./data/product_full_cp932.csv', 'wb') as f:
    f.write(buf.getvalue().decode('utf-8').encode('cp932'))

## P-096

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [9]:
df_product_full.write_csv('./data/product_full_nohead.csv', include_header=False)

## P-097

094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [10]:
pl.read_csv('./data/product_full.csv').head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,i64,str,i64,str,i64,str,i64,i64
"""P040101001""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",198,149
"""P040101002""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",218,164
"""P040101003""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",230,173
"""P040101004""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",248,186
"""P040101005""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",268,201


## P-098

096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [11]:
df_product_full.columns

['product_cd',
 'category_major_cd',
 'category_major_name',
 'category_medium_cd',
 'category_medium_name',
 'category_small_cd',
 'category_small_name',
 'unit_price',
 'unit_cost']

In [12]:
cols = ['product_cd',
 'category_major_cd',
 'category_major_name',
 'category_medium_cd',
 'category_medium_name',
 'category_small_cd',
 'category_small_name',
 'unit_price',
 'unit_cost']

pl.read_csv('./data/product_full_nohead.csv', has_header=False, new_columns=cols).head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,i64,str,i64,str,i64,str,i64,i64
"""P040101001""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",198,149
"""P040101002""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",218,164
"""P040101003""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",230,173
"""P040101004""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",248,186
"""P040101005""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",268,201


## P-099

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [13]:
df_product_full.write_csv('./data/product_full.tsv', separator='\t')

## P-100

099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。

> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [14]:
pl.read_csv('./data/product_full.tsv', separator='\t').head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,i64,str,i64,str,i64,str,i64,i64
"""P040101001""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",198,149
"""P040101002""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",218,164
"""P040101003""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",230,173
"""P040101004""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",248,186
"""P040101005""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",268,201
